## **University of Illinois Chicago**
CS 418 - Fall 2024 Team 5

## **Data-Driven Course Insights: Predicting Grade Trends**

## **Authors:**
| **Name**  | **Email** | **Github Handle** |
|---|---|---|
| Arlette Diaz | adiaz218@uic.edu | adiaz218 |
| Marianne Hernandez | mhern85@uic.edu | marhern19 |
| Nandini Jirobe | njiro2@uic.edu | nandinijirobe |
| Sharadruthi Muppidi | smuppi2@uic.edu | sharadruthi-uic |
| Sonina Mut | smut3@uic.edu | snina22 |
| Yuting Lu | lyuti@uic.edu | yutinglu103 |

**Github Repository Link: https://github.com/cs418-fa24/project-check-in-team-5**

## **Project Description**

This project is to predict course grade distributions and popularity rankings for upcoming semesters, enabling students to make informed decisions about their class selections. By shifting the focus from individual grade predictions to overall course outcomes, the project provides insights into course grading trends and demand. It uses clustering to rank courses based on student performance and popularity, and topic-based grouping to help students discover courses aligned with their interests, factoring in professor expertise and class attributes. This data-driven tool uncovers hidden patterns, aiding both students and academic planning.

## **Project Update**

### **Import Packages**

In [3]:
# import useful libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### **Part 1: Load Datasets**

### **Part 2: Data Cleaning**

### **Part 3: Exploratory Data Analysis**

### **Part 4: Data Visualizations**

### **Part 5: Machine Learning Analysis**

## **Reflection**

What is the hardest part of the project that you’ve encountered so far?What are your initial insights?Are there any concrete results you can show at this point? If not, why not?Going forward, what are the current biggest problems you’re facing?Do you think you are on track with your project? If not, what parts do you need todedicate more time to?Given your initial exploration of the data, is it worth proceeding with your project,why? If not, how are you going to change your project and why do you think it’sbetter than your current results?

## **Roles/Coordination (important)**

**Arlette Diaz:** 
<br>**Marianne Hernandez:** 
<br>**Nandini Jirobe:** 
<br>**Sharadruthi Muppidi:** 
<br>**Sonina Mut:** 
<br>**Yuting Lu:** 

## **Next Steps**